In [242]:
import sys
import preprocess_eegdata
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os
import mne
import mne_bids
from collections import defaultdict
import pandas as pd
import re
import shutil
import json

%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [172]:
# parent_dir = '../data/raw_data'
parent_dir = r'Z:\Darius\supracapacity\SCR_raw_data'
data_dir = r'Z:\Darius\supracapacity\data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
file_prefix='SCR_' # prefix to your vhdr files. Assuming it is in the format [prefix]_[number]


subject_dirs=[]  # if you want to analyze a specific subset of subjects
if len(subject_dirs) == 0:
    subject_dirs=sorted(glob('*',root_dir=parent_dir))


TRIAL_START_TIME=-0.4
TRIAL_END_TIME=1.4
BASELINE_TIME=(-.25,0)
REJECTION_TIME=[-0.25,1.0]

SRATE = 1000 # hz, will resample if different from 1k
FILTER_FREQS=(None,80) # None to not do lowpass

LINEAR_R2 = 0.3


og_event_dict={
    'trl_start':1,
    "attention/ss2/no_placeholders": 12,
    "attention/ss4/no_placeholders": 14,
    "attention/ss6/no_placeholders": 16,
    "attention/ss8/no_placeholders": 18,
    "attention/ss2/placeholders": 22,
    "attention/ss4/placeholders": 24,
    "attention/ss6/placeholders": 26,
    "attention/ss8/placeholders": 28,
    "memory/ss2/no_placeholders": 32,
    "memory/ss4/no_placeholders": 34,
    "memory/ss6/no_placeholders": 36,
    "memory/ss8/no_placeholders": 38,
    "memory/ss2/placeholders": 42,
    "memory/ss4/placeholders": 44,
    "memory/ss6/placeholders": 46,
    "memory/ss8/placeholders": 48,
    'delay_start':2,
    'attn_probe': 3,
    'delay_end':4,
}



event_dict=og_event_dict.copy()
event_code_dict={} # define event codes based on sequence

stim_conditions=[]
for key,ev in og_event_dict.items():
    if ev > 10:
        event_dict.update({key+'/TARGET':ev+1}) # add in keys for targets
        event_code_dict.update({ev:[1,ev,2,4,4]})
        event_code_dict.update({ev+1:[1,ev,2,3,2,4,4]})
        stim_conditions.extend([ev,ev+1])



POSITION_TO_TIMELOCK = 1 # which position (IN THE LIST ABOVE) to timelock to


EEG_TRIALS_DROP = {}   # TODO: make this a dict
EYE_TRIALS_DROP = {10:[0,1,2,3]} # edge case when we forgot to start the recording, drop certain trials
DROP_CHANNELS=[] #['Fp1','Fp2']


In [173]:
pre = preprocess_eegdata.Preprocess(
                                    data_dir=data_dir,
                                    experiment_name = 'supracapacity',
                                    trial_start=TRIAL_START_TIME,
                                    trial_end=TRIAL_END_TIME,
                                    event_names = og_event_dict,
                                    event_dict=event_dict,
                                    stim_conditions=stim_conditions,
                                    event_code_dict=event_code_dict,
                                    timelock_ix=POSITION_TO_TIMELOCK,
                                    baseline_time=BASELINE_TIME,
                                    rejection_time=REJECTION_TIME,
                                    no_et_spaces=False,
                                    drop_channels=DROP_CHANNELS,
                                    filter_freqs=FILTER_FREQS)


In [ ]:
for subject_number in subject_dirs:


    #####################
    #### IMPORT DATA ####
    #####################


    eeg,eeg_events = pre.import_eeg(parent_dir,subject_number,overwrite=True)
    eye,eye_events = pre.import_eyetracker(parent_dir,subject_number,overwrite=True)
    
    ########################################
    #### PREPROCESS EEG AND MAKE EPOCHS ####
    ########################################

    reref_index = mne.pick_channels(eeg.ch_names, ["TP9"])
    eeg.load_data().apply_function(pre.rereference_to_average, picks=["eeg"], reref_values=np.squeeze(eeg.get_data()[reref_index]))
    eeg.filter(*pre.filter_freqs, n_jobs=-1)
    epochs = pre.make_and_sync_epochs(eeg,eeg_events,eye,eye_events,srate=SRATE,eeg_trials_drop = EEG_TRIALS_DROP.get(subject_number, []),eye_trials_drop = EYE_TRIALS_DROP.get(subject_number, []))

    ###############################
    #### DO ARTIFACT REJECTION ####
    ###############################

    p2p=pre.artreject_slidingP2P(epochs,rejection_criteria={'eeg':100e-6,'eog':200},win=200,win_step=100)               # peak to peak in the window
    saccades = pre.artreject_step(epochs,rejection_criteria={'eyegaze':pre.deg2pix(0.5),'eog':50},win=80,win_step=10)   # saccades in EOG or eye tracking
    steps = pre.artreject_step(epochs,rejection_criteria={'eeg':60e-6},win=250,win_step=20)                             # steps (saccade like) in EEG

    absolute_value=pre.artreject_value(epochs,rejection_criteria={'eyegaze':pre.deg2pix(1), 'eeg':100e-6, 'eog':300})   # absolute value rejection
    linear_fit = pre.artreject_linear(epochs)                                                                           # linear fit (drift) rejection
    flatline = pre.artreject_flatline(epochs,rejection_criteria={'eeg':0,'eog':0,'eyegaze':0},flatline_duration=200)    # check for flatlines


    # combine rejection reasons
    rej_electrodes = p2p | saccades | steps | absolute_value | linear_fit | flatline
    rej_reasons = np.char.array(np.full(rej_electrodes.shape,'', dtype="<U30"))  # NOTE: dtype is important, must be >= the max possible str length
    rej_reasons[p2p] = 'P2P '
    rej_reasons[saccades] = rej_reasons[saccades] + 'SAC '
    rej_reasons[steps] = rej_reasons[steps] + 'STEP '
    rej_reasons[absolute_value] = rej_reasons[absolute_value] + 'ABS '
    rej_reasons[linear_fit] = rej_reasons[linear_fit] + 'LIN '
    rej_reasons[flatline] = rej_reasons[flatline] + 'FLAT '

    #################################
    #### SAVE DATA AS DERIVATIVE ####
    #################################

    pre.save_all_data(subject_number,epochs,rej_reasons)
